In [ ]:
import matplotlib.pylab as plt
import numpy as np

from landlab import HexModelGrid, imshow_grid
from landlab.components import FlowAccumulator, LinearDiffuser, FastscapeEroder, ChannelProfiler

In [ ]:
#grid = HexModelGrid((20, 10), 50)

nr = 50
nc = 30
dx = 50

np.random.seed(42)
grid1 = HexModelGrid((nr, nc), dx)
grid2 = HexModelGrid((nr, nc), dx)

initial_noise = np.random.rand(grid1.core_nodes.size)

z1 = grid1.add_zeros("topographic__elevation", at="node")
z1[grid1.core_nodes] += initial_noise

z2 = grid2.add_zeros("topographic__elevation", at="node")
z2[grid2.core_nodes] += initial_noise

plt.figure()
imshow_grid(grid1, z1, plot_name="baseline")
plt.figure()
imshow_grid(grid2, z2, plot_name="with change in K")

In [ ]:
K1 = grid1.add_ones("water_erodibility", at="node")
K1[:] = 0.0001

K2 = grid2.add_ones("water_erodibility", at="node")
K2[:] = 0.0001

In [ ]:
U = 0.0001
D = 0.01

K_min = 0.00001
K_max = 0.001

In [ ]:
fa1 = FlowAccumulator(grid1)
sp1 = FastscapeEroder(grid1, K_sp="water_erodibility")
ld1 = LinearDiffuser(grid1, linear_diffusivity=D)

fa2 = FlowAccumulator(grid2)
sp2 = FastscapeEroder(grid2, K_sp="water_erodibility")
ld2 = LinearDiffuser(grid2, linear_diffusivity=D)

In [ ]:
total_time = 1e6
dt = 100
nts = int(total_time / dt)

for i in range(nts):

    # topographic__steepest_slope is update in fa.r1s()
    fa1.run_one_step()
    fa2.run_one_step()

    # now we want to update K.
    # slope will vary between 0.0 0.021
    # When slope is 0 lets make K = 0.00001
    # When slope is 0.02 or above lets make K = 0.001

    slp2 = grid2.at_node["topographic__steepest_slope"].copy()
    slp2[slp2 > 0.02] = 0.02

    grid2.at_node["water_erodibility"][:] = K_min + (K_max - K_min) * slp2 / (
        0.02 - 0.0)

    sp1.run_one_step(dt)
    ld1.run_one_step(dt)
    z1[grid1.core_nodes] += U * dt

    sp2.run_one_step(dt)
    ld2.run_one_step(dt)
    z2[grid2.core_nodes] += U * dt

In [ ]:
vmax = max(z1.max(), z2.max())

plt.figure()
imshow_grid(grid1, z1, plot_name="baseline", vmax=vmax, cmap='gist_earth')
plt.figure()
imshow_grid(grid2,
            z2,
            plot_name="with change in K",
            vmax=vmax,
            cmap='gist_earth')

In [ ]:
plt.figure()
imshow_grid(grid1, "topographic__steepest_slope", plot_name="baseline")
plt.figure()
imshow_grid(grid2, "topographic__steepest_slope", plot_name="with change in K")

In [ ]:
cp1 = ChannelProfiler(grid1, number_of_watersheds=4, main_channel_only=True)
cp1.run_one_step()
plt.figure()
cp1.plot_profiles()
ax = plt.gca()
ax.set_title("baseline")

cp2 = ChannelProfiler(grid2, number_of_watersheds=4, main_channel_only=True)
cp2.run_one_step()
plt.figure()
cp2.plot_profiles()
ax = plt.gca()
ax.set_title("with change in K")

In [ ]:
plt.loglog(grid1.at_node["topographic__steepest_slope"],
           grid1.at_node["drainage_area"],
           ".",
           label="baseline")
plt.loglog(grid2.at_node["topographic__steepest_slope"],
           grid2.at_node["drainage_area"],
           ".",
           label="with change in K")

plt.legend()